In [ ]:
import tensorflow as tf
import pandas as pd
import pathlib
import os
import cv2
import numpy as np
import keras

from PIL import Image

from commonfun import create_dataset, apply_mask_overlay
from model_func import build_unet_model

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

CAMVID_COLOR_MAP = {
    0:  (128, 128, 128),  # Sky      	# 하늘
    1:  (128, 0, 0),      # Building 	# 건물
    2:  (192, 192, 128),  # Pole			# 전봇대, 기둥
    3:  (128, 64, 128),   # Road			# 도로
    4:  (0, 0, 192),      # Pavement	# 보도
    5:  (128, 128, 0),    # Tree			# 나무
    6:  (192, 128, 128),  # SignSymbol # 교통표지판
    7:  (64, 64, 128),    # Fence		# 울타리
    8:  (64, 0, 128),     # Car			# 자동차
    9:  (64, 64, 0),      # Pedestrian # 보행자
    10: (0, 128, 192),    # Bicyclist	# 자전거 타는 사람
    11: (255, 255, 255)   # None
}

def applyColorMap(mask_arr, colormap=CAMVID_COLOR_MAP):
  print('mask_arr : ', mask_arr.shape)
  color_arr = np.array([colormap[i] for i in range(12)], dtype=np.uint8)
  mask_arr = color_arr[mask_arr]

  return mask_arr

In [ ]:
test_df = pd.read_csv('./CamVid/test.txt', sep=' ', header=None)
test_df.columns = ['image', 'mask']

test_df['image'] = test_df['image'].str.replace('/SegNet', '.')
test_df['mask'] = test_df['mask'].str.replace('/SegNet', '.')

test_df

In [ ]:
val_df = pd.read_csv('./CamVid/val.txt', sep=' ', header=None)
val_df.columns = ['image', 'mask']
val_df['image'] = val_df['image'].str.replace('/SegNet', '.')
val_df['mask'] = val_df['mask'].str.replace('/SegNet', '.')
val_df

In [ ]:
image_paths = test_df['image'].values.tolist()
mask_paths = test_df['mask'].values.tolist()

In [ ]:
val_image_paths = val_df['image'].values.tolist()
val_mask_paths = val_df['mask'].values.tolist()

In [ ]:
train_ds = create_dataset(image_paths, mask_paths)

train_ds

In [ ]:
val_ds = create_dataset(val_image_paths, val_mask_paths)
val_ds

In [ ]:
model = build_unet_model(input_shape=(256, 256, 1), num_class=12)

model.compile(
   loss='sparse_categorical_crossentropy',
   optimizer='adam',
   metrics=['accuracy']
)

model.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model_check = keras.callbacks.ModelCheckpoint(
   'unet_best.keras',
   monitor='val_loss', 
   save_weights_only=False,
   mode='min',
   verbose=1
   )

model.fit(train_ds,
          epochs=1,
          validation_data=val_ds,
          callbacks=[early_stop, model_check]
   )